<a href="https://www.kaggle.com/code/yutodennou/model-futurocoinsaga-prophet?scriptVersionId=176227015" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🗃️Import Library

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install prophet
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import io

# 🔍Import data

In [ ]:
df = pd.read_csv("/kaggle/input/futurocoin-saga-a-200-year-cryptocurrency-odyssey/futurocoin_dataset.csv")
df.shape

In [ ]:
df.head(20)

In [ ]:
df = df.rename(columns={'Date': 'ds','Trading USD Value': 'y'})

In [ ]:
df.plot()
plt.xlabel('index')
plt.ylabel('y')
plt.show()

# 📈Make Prophet Model

In [ ]:
model = Prophet()
model.fit(df)

In [ ]:
# Set the Period for Train
future_df = model.make_future_dataframe(365)
# Predict
forecast_df = model.predict(future_df)
forecast_df.head()

In [ ]:
# Sellect the Seasonality
model = Prophet(daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=True)
model.fit(df)
future_df = model.make_future_dataframe(365)
forecast_df = model.predict(future_df)
model.plot(forecast_df)
plt.show()

In [ ]:
# Show Trend, Seasonal, Period Components
model.plot_components(forecast_df)
plt.show()

In [ ]:
# Changepoint
changepoints_df = df.loc[model.changepoints.index]
changepoints_df['ds'] = changepoints_df['ds'].astype(str)
changepoints_df['delta'] = model.params['delta'].ravel()

# sns.set(style='whitegrid')
# ax = sns.factorplot(x='ds', y='delta', data=changepoints_df, kind='bar', height=6, aspect=2)
# ax.set_xticklabels(rotation=90)

In [ ]:
fig = model.plot(forecast_df)

tmp = changepoints_df[abs(changepoints_df['delta']) >= 0.4]
date_changepoints = tmp['ds'].astype('datetime64[ns]').reset_index(drop=True)

for dt in date_changepoints:
    plt.axvline(dt, ls='--', lw=2)
plt.show()

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='730 days', period='180 days', horizon = '365 days')

#評価値計算
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)

#評価値描画
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')